<a href="https://colab.research.google.com/github/ericzhang789/mortgage-tools/blob/bokeh/mortgage_payment_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. Import Dependencies
%%capture
!pip install pandas numpy numpy_financial matplotlib bokeh ipywidgets
import math
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
from ipywidgets import interact

from bokeh.layouts import column, row
from bokeh.io import output_notebook, show 
from bokeh.models import LabelSet, ColumnDataSource, HoverTool
from bokeh.models.widgets import DataTable, TableColumn, NumberFormatter
from bokeh.plotting import figure, output_file, curdoc
from bokeh.palettes import Pastel2_4
from bokeh.transform import dodge
output_notebook()

In [ ]:
#@title 2. Define Functions

#Global Variables
cmhc_rates = {5:0.04, 10:0.031, 15:0.028, 20:0.00}
PST = 0.08 #PST not incl in calculation, it is paid upfront

#create payment schedules
def payment_schedule(purchase_price, percent_down, interest_rate, amortization, pmt_freq):
  #apply downpayment
  mortgage_amount = purchase_price*(1-(percent_down/100))
  
  #add mortgage insurance if applicable
  mortgage_amount += mortgage_amount*cmhc_rates[percent_down]

  #cleanse interest rate
  interest_rate = interest_rate/100

  payment_frequency = pmt_freq

  #payment schedule calculation
  pmt = npf.pmt(interest_rate/payment_frequency, payment_frequency*amortization, mortgage_amount)
  period = np.arange(payment_frequency*amortization)+1
  ipmt = npf.ipmt(interest_rate/payment_frequency, period, amortization*payment_frequency, mortgage_amount)
  ppmt = npf.ppmt(interest_rate/payment_frequency, period, amortization*payment_frequency, mortgage_amount)
  #np.allclose(ipmt+ppmt, pmt)

  principal = mortgage_amount
  pcpl = []
  year = []
  fmt = '{0:2d} {1:8.2f} {2:8.2f} {3:8.2f}'

  for payment in period:
    index = payment - 1
    principal = principal + ppmt[index]
    pcpl.append(principal)
    year.append(math.ceil(period[index]/payment_frequency))
    #print(fmt.format(payment, ppmt[index], ipmt[index], principal))

  #format nicely
  pcpl = np.around(pcpl, 2)
  pmt = np.around(pmt, 2)
  ipmt = np.around(ipmt, 2)
  ppmt = np.around(ppmt, 2)
  
  pmt_cumsum = np.cumsum(np.full(len(period),abs(pmt)))
  ppmt_cumsum = np.cumsum(np.full(len(period),abs(ppmt)))
  ipmt_cumsum = np.cumsum(np.full(len(period),abs(ipmt)))
  #arrange in dict then return df
  pmt_dict = {'pmt_num': abs(period), 'year': year, 
              'pmt': abs(pmt), 'ppmt': abs(ppmt),'ipmt': abs(ipmt),
              'pmt_cumsum': pmt_cumsum, 'ppmt_cumsum': ppmt_cumsum, 'ipmt_cumsum': ipmt_cumsum, 
              'pcpl_rem': abs(pcpl)}
  return pd.DataFrame(data = pmt_dict)

# #plotting function
# def plot_payment_schedule():
#   #set up which scenarios to plot
#   group = [None, None, None, None]

#   for scen in range(len(scenario_bool)):
#     #plot only if scenario selected and df exists
#     if scenario_bool[scen] == True and pmt_sched[scen] is not None:
#       #clean data for stacked and grouped plotting
#       #last payment of the year
#       group[scen] = pmt_sched[scen].groupby('year').max('pmt_num')
#       #mean payment per year
#       #group = pmt_sched[0].groupby('year').mean().round(2)

#   #pmt_str = str(group.pmt.max()) + ' ' + payment_frequency

#   plot_df = pd.concat([group[scen] for scen in range(len(scenario_bool)) if scenario_bool[scen] is True], 
#                     axis = 1, keys = [1 + i for i, x in enumerate(scenario_bool) if x])

#   x_grp = []
#   for idx in plot_df.index:
#     for scen in list(set(plot_df.columns.get_level_values(0))):
#       x_grp.append((str(scen), str(idx)))

#   TOOLTIPS = [('Principal', '@ppmt{$0.2f}'), ('Interest', '@ipmt{$0.2f}')]

 
#   p1 = figure(x_range = FactorRange(*x_grp),
#             plot_width = 900, x_axis_label='Year', y_axis_label='Payment Amount per Period ($)', 
#             title = 'Mortgage Payment components at Year End', #($' + pmt_str+')',
#             tooltips=TOOLTIPS
#             )
#   p1.vbar_stack(['ppmt', 'ipmt'], x='x', source = group, width=0.9)
#   p1.y_range.start = 0
#   p1.y_range.range_padding = 0.25
#   p1.x_range.start = 0

#   p1.legend.location = 'top_left'
#   p1.legend.orientation = 'horizontal'

#   TOOLTIPS = [('Year', '@year'),('Principal Remaining', '@pcpl_rem{$0.2f}')]

#   p2 = figure(plot_width = 900, x_axis_label = 'Year', y_axis_label = 'Remaining Principal ($)', 
#               x_range = p1.x_range, y_range = p1.y_range,
#               title = 'Remaining Principal at Year End', tooltips=TOOLTIPS)

#   p2.line(x='year', y='pcpl_rem', source = group ,line_width = 2)
#   p2.circle(x='year', y='pcpl_rem', source = group, fill_color="white", size=8)
#   p2.y_range.start = 0
#   p2.y_range.range_padding = 0.25
#   p2.left[0].formatter.use_scientific = False
#   p2.x_range.start = 0

#   return row(p1, p2)


def get_summary_df():
  rows_list = []

  for scen in range(len(pmt_sched)):
    my_dict = {}
    if pmt_sched[scen] is not None:
      idx_max = pmt_sched[scen].loc[pmt_sched[scen]['year']<=term_years].index.max()
      my_dict.update(pmt_sched[scen].iloc[idx_max])
      my_dict['scenario'] = 'Scenario ' + str(scen+1)
      rows_list.append(my_dict)

  return pd.DataFrame(rows_list).set_index('scenario')

def show_term_summary():
  df = get_summary_df()
  #----------------------------plot---------------------------
  totals = ['Mortgage Balance', 'Total Paid', 'Principle Paid', 'Interest Paid']
  scenarios = list(df.index) #0-index

  data = {'totals': totals}

  for val in df.index:
    data[val] = df.loc[val, ['pcpl_rem', 'pmt_cumsum', 'ppmt_cumsum', 'ipmt_cumsum']].round(0).tolist()

  source = ColumnDataSource(data=data)

  TOOLTIPS = ('$name: @$name{$,}')

  p = figure(x_range=totals, plot_width=1000, tooltips=TOOLTIPS,
            title='Balances at Term Maturity (' + str(term_years) + ' years)')

  for idx, val in enumerate(df.index):
    p.vbar(x=dodge('totals', -0.30 + idx*0.20, range=p.x_range), top = val, width = 0.15, source = source, color=Pastel2_4[idx], legend_label=val, name=val)

  p.y_range.start=0
  p.xgrid.grid_line_color = None
  p.left[0].formatter.use_scientific = False

  p.legend.location = 'top_right'
  p.legend.click_policy = 'hide'

  #------------------------datatable-------------------------
  data = {None}
  data = df.T.loc[['pmt', 'pmt_cumsum', 'ppmt_cumsum','ipmt_cumsum','pcpl_rem']].round(2)

  data['display'] = ['Payment', 'Total Paid', 'Principle Paid', 'Interest Paid', 'Mortgage Balance']

  source = ColumnDataSource(data)

  columns = [TableColumn(field='display', title='', width=500, sortable=False)]

  for col in data.columns:
    if col != 'display':
      columns.append(TableColumn(field=col, formatter=NumberFormatter(format='$0,000', text_align='right'), sortable=False))

  tbl = DataTable(source=source,columns=columns, width=400)
  tbl.index_position = None

  return row(p, tbl)
  #show(row(p, tbl))

In [ ]:
#@markdown Enter mortgage terms and run to generate plots and table

#Mortgage details:
scenario_bool = [True, True, True, True] #show data for scenario?
purchase_price = [500000, 500000, 500000, 500000]
percent_down = [10, 10, 15, 20] #doesn't work if < 10% down
interest_rate = [1.68, 0.99, 1.68, 1.74]
term_years = 5 #all must be same
amortization = [25, 25, 25, 25]
payment_frequency = 12 #make this a radiobox for all scenarios
#export_csv

#instantiate array to store pandas df of payment schedules for each scenario
pmt_sched = [None, None, None, None]

#generate payment schedules
for scen in range(len(scenario_bool)):
  if scenario_bool[scen] == True:
    #get payment schedule for populated scenarios
    pmt_sched[scen] = payment_schedule(purchase_price[scen], percent_down[scen],
                                      interest_rate[scen], amortization[scen],
                                        payment_frequency)

#generate term summary plot and table
get_summary_df()
show(show_term_summary())
